(Uncompleted) Notebook for training and testing the baseline neural network model using adjacency matrices of the AST format of the buffer overflow datapoints. In order for the matrices to be fed into the neural network, they must all be of the same dimensions. We currently pick a subset of the data with small-ish AST matrices (614x614)

# Colab Setup

In [1]:
!mkdir -p /project/data && cd /project/data && wget -O adj.pickle https://github.com/dj311/uob-summer-project/raw/master/data/adj.pickle
!mkdir -p /project/data && cd /project/data && wget -O buffer_overflow_data.csv.gz https://github.com/dj311/uob-summer-project/raw/master/data/buffer_overflow_data.csv.gz
!mkdir -p /project/code
%cd /project/code

--2019-07-26 15:59:33--  https://github.com/dj311/uob-summer-project/raw/master/data/adj.pickle
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/dj311/uob-summer-project/master/data/adj.pickle [following]
--2019-07-26 15:59:33--  https://media.githubusercontent.com/media/dj311/uob-summer-project/master/data/adj.pickle
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1320521199 (1.2G) [application/octet-stream]
Saving to: ‘adj.pickle’

adj.pickle          100%[===================>]   1.23G   252MB/s    in 5.0s    

2019-07-26 15:59:53 (250 MB/s) - ‘adj.pickle’ saved [1320521199/132052119

# Import & Preprocess Dataset

First we import the data from the [previous notebook](./adjacency_matrix.ipynb).


In [2]:
import tensorflow as tf
import keras
import sklearn
import numpy as np
import pickle
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.model_selection import train_test_split


run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
np.random.seed(1248)

tf.logging.set_verbosity(tf.logging.INFO)

Using TensorFlow backend.


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('../data/buffer_overflow_data.csv.gz')

In [0]:
labels = data.copy()
del labels['Unnamed: 0']
del labels['Unnamed: 0.1']
del labels['filename']
del labels['code']
del labels['flaw']
del labels['flaw_loc']
labels = labels.drop_duplicates().sort_values('testcase_ID').reset_index(drop=True)

In [0]:
with open("../data/adj.pickle",'rb') as f:
    adj = pickle.load(f)

In [0]:
adj = adj.rename(columns={0: 'testcase_ID', 1: 'matrix'})

In [0]:
adj_df = pd.merge(labels, adj, on='testcase_ID')

In [0]:
adj_df = adj_df[['testcase_ID', 'matrix', 'bug']]

Next, find out the maximum size of an adjacency matrix, then convert all matrices to have the same dimension.

In [0]:
adj_df['matrix_size'] = adj_df.matrix.apply(lambda x: x.shape[0])

In [11]:
adj_df['matrix_size'].describe()

count    21502.000000
mean      2186.138778
std       7239.752920
min          4.000000
25%        349.000000
50%        396.000000
75%        614.000000
max      44401.000000
Name: matrix_size, dtype: float64

So we know that 75% of the dataset has a matrix size <= 614, which is approximately 18000 datapoints. Picking the full dataset would require matrices of dimension 44401x44401 which require 15 gb of memory each. This is isn't feasible, however picking matrices of size 614x614 or less gives a per matrix size of 3.38mb - far more manageable!

In [12]:
matrix_size = 614
adj_df = adj_df[adj_df['matrix_size'] <= matrix_size]
len(adj_df)

16128

In [13]:
adj_df['matrix'].values[0]

<395x395 sparse matrix of type '<class 'numpy.int64'>'
	with 788 stored elements in Compressed Sparse Row format>

In [0]:
def matrix_size_corrector(matrix, size):
    '''Pads matrix with zeros to the desired size'''
    
    rows, columns = matrix.shape[0], matrix.shape[1]
    
    row_corrector = csr_matrix((size-rows, rows))
    col_corrector = csr_matrix((size, size-columns))

    matrix = vstack([matrix, row_corrector])
    matrix = hstack([matrix, col_corrector])

    matrix = matrix.astype(np.int)
    
    return matrix 

In [0]:
adj_df['matrix'] = adj_df['matrix'].apply(lambda m: matrix_size_corrector(m, matrix_size))

Now we have a dataframe for each testcase with a sparse representation of its AST in the matrix column, each normalised to matrix_size x matrix_size in size.

Now, to input this into our model, we create three numpy arrays, linked by their index:
  1. `testcase_ids` gives the testcase ID of each row
  2. `adjacency_matrices` is an array of 2D adjacency matrices
  3. `labels` tells us whether each row contains a bug or not
  
  



In [0]:
testcase_ids = adj_df['testcase_ID'].values
adjacency_matrices = adj_df['matrix'].values
labels = adj_df['bug'].values

Storing all of these matrices in a dense representation at once might cause memory issues. To avoid this, we write a class which generates dense matrices for each of the training batches. 

We also perform the element wrapping as part of this process (since we can't perform it on the sparse arrays, I think).

In [0]:
class SparseToDenseGenerator(keras.utils.Sequence):

    def __init__(self, sparse_matrices, labels, batch_size):
        self.sparse_matrices = sparse_matrices
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.sparse_matrices) / float(self.batch_size)))

    def __getitem__(self, batch_num):
        start_index = batch_num * self.batch_size
        end_index = (batch_num + 1) * self.batch_size
        
        batch_sparse = self.sparse_matrices[start_index:end_index]
        batch_labels = self.labels[start_index:end_index]
        
        batch_dense = np.array([sparse_matrix.todense() for sparse_matrix in batch_sparse])
        
        # TODO: move this somewhere better
        # Conv2D requires an extra dimension for "channels", so we need to convert our data from
        # the shape (batch_size, matrix_rows, matrix_columns)
        # to (batch_size, matrix_rows, matrix_columns, 1)
        batch_dense = np.reshape(batch_dense, batch_dense.shape + (1, ))

        return batch_dense, np.array(batch_labels)

Finally, we generate the train and test splits:

In [0]:
x_train, x_test, y_train, y_test = train_test_split(adjacency_matrices, labels, test_size=0.2)

# Model

In [0]:
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, Conv2D, GlobalMaxPooling2D, MaxPooling2D, Convolution2D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import RMSprop, Adadelta, Adam
from keras.utils import np_utils
from keras import backend as K

In [20]:
batch_size = 96
epochs = 5
num_samples = len(labels)

datapoint_shape = (matrix_size, matrix_size, )
batch_shape = (batch_size, ) + datapoint_shape

steps_per_epoch = int(np.ceil(num_samples/batch_size))

kernel_size = (2, 2)
strides = max(kernel_size[0] // 3, 1)

batch_size, epochs, num_samples, datapoint_shape, batch_shape, steps_per_epoch, kernel_size, strides

(96, 5, 16128, (614, 614), (96, 614, 614), 168, (2, 2), 1)

In [21]:
model = Sequential()

model.add(Conv2D(
    data_format='channels_last',
    input_shape=(matrix_size, matrix_size, 1),
    filters=32,
    kernel_size=kernel_size,
    strides=strides,
))
model.add(Activation('relu'))
model.add(MaxPooling2D(kernel_size, padding='same'))

model.add(Conv2D(
    data_format='channels_last',
    input_shape=(matrix_size, matrix_size, 1),
    filters=32,
    kernel_size=kernel_size,
    strides=strides,
    activation='relu',
))
model.add(Activation('relu'))
model.add(MaxPooling2D(kernel_size, padding='same'))

model.add(Conv2D(
    data_format='channels_last',
    input_shape=(matrix_size, matrix_size, 1),
    filters=32,
    kernel_size=kernel_size,
    strides=strides,
    activation='relu',
))
model.add(Activation('relu'))
model.add(MaxPooling2D(kernel_size, padding='same'))

model.add(Flatten())

model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=32, activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

W0726 16:00:17.741810 140183665493888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 16:00:17.757972 140183665493888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 16:00:17.761829 140183665493888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 16:00:17.780237 140183665493888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0726 16:00:17.867340 140183665493888 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 613, 613, 32)      160       
_________________________________________________________________
activation_1 (Activation)    (None, 613, 613, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 307, 307, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 306, 306, 32)      4128      
_________________________________________________________________
activation_2 (Activation)    (None, 306, 306, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 153, 153, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 152, 152, 32)      4128      
__________

In [22]:
training_batch_generator = SparseToDenseGenerator(x_train, y_train, batch_size)

model.fit_generator(
    generator=training_batch_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch
)

Epoch 1/5
168/168 [==============================] - 2402s 14s/step - loss: 0.5190 - acc: 0.7042
Epoch 2/5
168/168 [==============================] - 2368s 14s/step - loss: 0.1684 - acc: 0.9265
Epoch 3/5
168/168 [==============================] - 2449s 15s/step - loss: 0.0734 - acc: 0.9797
Epoch 4/5
168/168 [==============================] - 2473s 15s/step - loss: 0.0348 - acc: 0.9912
Epoch 5/5
168/168 [==============================] - 2472s 15s/step - loss: 0.0201 - acc: 0.9944


In [0]:
# Our "Ideal" model that we currently can't get to run.

# nb_filters = 100
# pool_size = 3
# input_shape = datapoint_shape

# b_model = Sequential()

# b_model.add(Convolution2D(nb_filters, (100, 100),
#                         padding='same',
#                         input_shape=(matrix_size, matrix_size, 1))) # 卷积层1
# b_model.add(Activation('sigmoid')) #激活层
# b_model.add(Convolution2D(nb_filters, (10, 10))) #卷积层2
# b_model.add(Activation('sigmoid')) #激活层
# b_model.add(MaxPooling2D(pool_size=pool_size)) #池化层
# b_model.add(Dropout(0.25)) #神经元随机失活
# b_model.add(Convolution2D(nb_filters, (10, 10))) #卷积层2
# b_model.add(Activation('sigmoid')) #激活层
# b_model.add(MaxPooling2D(pool_size=pool_size)) #池化层
# b_model.add(Dropout(0.25)) #神经元随机失活

# b_model.add(Convolution2D(nb_filters, (10, 10))) #卷积层2
# b_model.add(Activation('sigmoid')) #激活层
# b_model.add(MaxPooling2D(pool_size=pool_size)) #池化层
# b_model.add(Dropout(0.25)) #神经元随机失活

# #model.add(Flatten()) #拉成一维数据
# b_model.add(Dense(128)) #全连接层1
# b_model.add(Activation('sigmoid')) #激活层
# b_model.add(Dropout(0.5)) #随机失活

# # model.add(Flatten()) #拉成一维数据
# # model.add(Activation('sigmoid')) #激活层
# # model.add(Dropout(0.5)) #随机失活

# b_model.add(Dense(1)) #全连接层2
# b_model.add(Activation('sigmoid')) #Softmax评分


# b_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# b_model.summary()

# Evaluation

In [0]:
 import pickle

In [0]:
with open('../data/adjacency-matrix-model-binary','wb') as f:
    pickle.dump(model,f)

In [32]:
test_batch_generator = SparseToDenseGenerator(x_test, y_test, batch_size)
model.evaluate_generator(
    generator=test_batch_generator)

[0.021008730195670563, 0.9944203250983156]

In [0]:
import seaborn as sns
from sklearn import metrics
import matplotlib

In [0]:
y_predict = model.predict(x_test)

In [0]:
import numpy as np

In [0]:
confusion_matrix = pd.DataFrame(
    data=metrics.confusion_matrix(y_test, np.rint(y_predict)),
)

confusion_figure, confusion_axes = matplotlib.pyplot.subplots()
confusion_figure.set_size_inches(15, 12)
confusion_axes.set_title(
    'Confusion matrix showing the frequency of \n'
    'correct and incorrect bug classification predictions.'
    '\n\n'  # hack to avoid overlap with x-axis labels below
)
confusion_axes.xaxis.tick_top()  # move x-axis labels to top of matrix
_ = sns.heatmap(
    confusion_matrix,
    annot=True,
    fmt="d",
    cmap=sns.color_palette("Blues"),
    vmin=0,
    ax=confusion_axes,
)

In [0]:
from sklearn.metrics import roc_curve
fpr_keras, tpr_keras, thresholds_keras = roc_curve((y_test.values+0).argmax(axis=1)-5, y_predict.argmax(axis=1)-5)

In [0]:
from sklearn.metrics import auc
import matplotlib.pyplot as plt
auc_keras = auc(fpr_keras, tpr_keras)

In [0]:
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [0]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test, np.rint(y_predict)).flatten().tolist()fpr_nn = fp/(fp+tp)
fnr_nn = fn/(fn+tn)